https://towardsdatascience.com/inverse-physics-informed-neural-net-3b636efeb37e

https://github.com/w-wojtak/PINNs-and-iPINNs-Pytorch/blob/main/iPINN_RLC_Pytorch.ipynb

In [ ]:
import datetime
import os

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)
np.random.seed(42)

In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self, parameters):
        super(NeuralNetwork, self).__init__()
        [nn_layers, act_fun] = parameters

        # Define a dictionary for activation functions
        af_list = {
            'tanh': nn.Tanh(),
            'sigmoid': nn.Sigmoid(),
            'relu': nn.ReLU(),
            'gelu': nn.GELU()
        }
        self.activation_function = af_list.get(act_fun, None)

        # Check if activation function is provided
        if self.activation_function is None:
            raise ValueError(f"Activation function '{act_fun}' is not supported.")

        # Create layers dynamically based on nn_layers
        self.layers = nn.ModuleList()
        for i in range(len(nn_layers) - 1):
            self.layers.append(nn.Linear(nn_layers[i], nn_layers[i + 1]))
            if i < len(nn_layers) - 2:  # No activation function after the last layer
                self.layers.append(self.activation_function)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [ ]:
def pred(net_model, t):
    model_return = net_model(t)
    return model_return

def squared_difference(input, target):
    return (input - target) ** 2

# t_range, R, L, C, VC_0 = ode_parameters  
def calculate_analytical_solution(ode_parameters, scenario, ni):
    t_range, _, _, _, VC_0 = ode_parameters
    i_calc = np.zeros(ni)
    t_test = np.linspace(t_range[0], t_range[1], ni)

    R = 1.2 # resistance, ohm
    L = 1.5 # inductance, H
    C = 0.3 # capacitance, F
    VC_0 = 12.0 # volts, initial capacitor voltage
    alpha = R / (2 * L)

    for i in range(ni):
        i_calc[i] = 5.57 * np.exp(-0.4 * t_test[i]) * np.sin(1.44 * t_test[i])

    return t_test, i_calc


In [ ]:
def train(ode_parameters, train_parameters, nn_parameters, t_f, u_test_pred):
    # loading parameters
    [scenario, ni, opt, max_epochs, min_loss, learning_rate] = train_parameters
    [t_range, R, L, C, VC_0] = ode_parameters

    # initial condition - same as for PINN
    u_0 = [0.0];
    t_i = torch.FloatTensor(np.array(t_range[0]).reshape(-1, 1)) #torch.FloatTensor([[0.0]])
    u_i = torch.FloatTensor(np.array(u_0).reshape(-1, 1)) #torch.FloatTensor([[0.0]])

    #  f'(t) initial conditions - same as for PINN
    u_i2 = torch.FloatTensor(np.array(VC_0).reshape(-1, 1)) #u_i2: tensor([[12.]])

    t_i.requires_grad = True
    t_f.requires_grad = True

    net_model = NeuralNetwork(nn_parameters)

    train_R = torch.nn.Parameter(torch.FloatTensor([R]), requires_grad=True)
    net_model.register_parameter('R', train_R)

    train_L = torch.nn.Parameter(torch.FloatTensor([L]), requires_grad=True)
    net_model.register_parameter('L', train_L)

    train_C = torch.nn.Parameter(torch.FloatTensor([C]), requires_grad=True)
    net_model.register_parameter('C', train_C)

    # choose optimizer
    if opt == 1:
        optimizer = torch.optim.Adam([{'params': net_model.parameters()}], lr=learning_rate)
    elif opt == 2:
        optimizer = torch.optim.SGD([{'params': net_model.parameters()}], lr=learning_rate)
    else:
        optimizer = torch.optim.LBFGS([{'params': net_model.parameters()}], lr=learning_rate)

    epoch = 0
    loss = 10
    loss_record = np.empty([0, 3])
    rlc_record = np.empty([0, 3])
    plt.ion()
    print('------------------------Neural network------------------------------------')
    print(net_model)
    print('----------------------------Optimizer--------------------------------------')
    print(optimizer)
    #  -----------   start training   ------------
    starttime_train = datetime.datetime.now()
    formatted_time = starttime_train.strftime("%Y-%m-%d %H:%M:%S")
    print('------------------------Start training:{}---------------------'.format(formatted_time))

    while epoch < max_epochs and loss > min_loss:
        def closure():
            # u_i refers to the initial condition, u_f to the solution
            u_i_pred = pred(net_model, t_i) #초기조건 t_i에서 net_model을 사용해 예측
            u_f_pred = pred(net_model, t_f) #도메인 내 임의의 점 t_f에서 net_model을 사용해 예측

            u_i_pred_dt = torch.autograd.grad(u_i_pred.sum(), t_i, create_graph=True)[0] #초기조건에서의 예측값에 대한 시간 미분
            u_f_pred_dt = torch.autograd.grad(u_f_pred.sum(), t_f, create_graph=True)[0] # 도메인 내 임의의 점에서 예측값에 대한 시간 미분
            u_f_pred_dtt = torch.autograd.grad(u_f_pred_dt.sum(), t_f, create_graph=True)[0] #도메인 내 임의의 점에서 예측값에 대한 2차 시간 미분

            f = (train_L * u_f_pred_dtt) + (train_R * u_f_pred_dt) + (u_f_pred / train_C)
            
            # get the three loss components
            loss_1 = torch.mean(squared_difference(u_i_pred, u_i))
            loss_2 = torch.mean(squared_difference(f, torch.zeros_like(t_f)))
            loss_3 = torch.mean(squared_difference(u_i_pred_dt, (u_i2 / train_L)))
            loss_4 = torch.mean(squared_difference(u_f_pred, u_test_pred))

            loss_total = loss_1 + loss_2 + loss_3 + loss_4
            #TODO!


In [ ]:
dirs = ['./model', './data', './figures']
for dir in dirs:
    if not os.path.exists(dir):
        os.makedirs(dir)

scenario = 0 # 0: under-damped, 1: critically-damped, 2: over-damped

# Load data
base_filename = './data/pinn_data'

# Determine the filename based on the scenario
if scenario == 0:
    filename = base_filename + '_under-damped.pt'
elif scenario == 1:
    filename = base_filename + '_critically-damped.pt'
elif scenario == 2:
    filename = base_filename + '_over-damped.pt'
else:
    raise ValueError("Invalid scenario value. Scenario should be 0, 1, or 2.")

# Load the tensor from the file
pinn_data = torch.load(filename)

# Extract t_f and u_test_pred from the loaded tensor
# t_f : time
# u_test_pred
t_f, u_test_pred = pinn_data[:, 0], pinn_data[:, 1]
#pinn_data.size() -> torch.Size([50, 2])
t_f = t_f.unsqueeze(1)
u_test_pred = u_test_pred.unsqueeze(1)

t_range = [0.0, 6.0]
VC_0 = 12.0
R, L, C = 0.1, 0.1, 0.1
ode_parameters = [t_range, R, L, C, VC_0]

ni = 50
optimizer = 0  # 0: L-BFGS 1: Adam 2: SGD
max_epochs = 5000
min_loss = 1e-8
learning_rate = 0.01
train_parameters = [scenario, ni, optimizer, max_epochs, min_loss, learning_rate]

test_parameters = [scenario, ni]

# Neural networks parameters
nn_layers = [1, 128, 128, 1]  # neural networks layers
act_fun = 'gelu'
nn_parameters = [nn_layers, act_fun]

train(ode_parameters, train_parameters, nn_parameters, t_f, u_test_pred)